In [1]:
normalizedScores = {("b6e71c034fa5e34b5d8a9199208d53cb", 76): 0.3, ("b6e71c034fa5e34b5d8a9199208d53cb", 176): 0.85, ("b6e71c034fa5e34b5d8a9199208d53cb", 85): 0.4}
powers = {176:0.5, 76:0.9, 85:0.8}
def getIntegratedAdomniScore(normalizedScores, powers):
    integratedAdomniScore = 0
    for locationhash_and_segId in normalizedScores:
        integratedAdomniScore = integratedAdomniScore + normalizedScores[locationhash_and_segId] * powers[locationhash_and_segId[1]] 
    return integratedAdomniScore/len(normalizedScores)

getIntegratedAdomniScore(normalizedScores, powers)

0.3383333333333334

In [2]:
import json
with open('DynamoDBtoCSV-master/config.json') as f:
    Tuo_authentications = json.load(f)

In [5]:
import boto3
def run_query(query, s3_output):
    client = boto3.client('athena',
                           aws_access_key_id = Tuo_authentications['accessKeyId'],
                           aws_secret_access_key = Tuo_authentications['secretAccessKey'],
                           region_name = Tuo_authentications["region"])
    response = client.start_query_execution(
        QueryString = query,
        QueryExecutionContext = {
          'Database': 'default'
        },
        ResultConfiguration = {
            'OutputLocation': s3_output,
            'EncryptionConfiguration': {
                'EncryptionOption': 'SSE_S3'
            }
        }
        
    )
    print('Execution ID: ' + response['QueryExecutionId'])
    return response


s3_output = 's3://tuo-bucket/playing'

query = "select * from adomni_audience_segment where id='{ID}';"

params = {"ID":"740"}

QueryResponse = run_query(query.format(**params), s3_output)
print(QueryResponse)
QueryExecutionId = QueryResponse['QueryExecutionId']

Execution ID: 90692063-f50d-46da-8c14-beddd0d12b3f
{'QueryExecutionId': '90692063-f50d-46da-8c14-beddd0d12b3f', 'ResponseMetadata': {'RequestId': '45e226b5-d1c3-43aa-b646-072ca3385bb6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1', 'date': 'Sun, 13 Oct 2019 15:24:59 GMT', 'x-amzn-requestid': '45e226b5-d1c3-43aa-b646-072ca3385bb6', 'content-length': '59', 'connection': 'keep-alive'}, 'RetryAttempts': 0}}


In [1]:
import json
with open('DynamoDBtoCSV-master/config.json') as f:
    Tuo_authentications = json.load(f)

import boto3
from boto3.dynamodb.conditions import Key, Attr

dynamodb = boto3.resource('dynamodb',
                           aws_access_key_id = Tuo_authentications['accessKeyId'],
                           aws_secret_access_key = Tuo_authentications['secretAccessKey'],
                           region_name = Tuo_authentications["region"]
)


table = dynamodb.Table('machine_learning')

response = table.scan()
# print(response)
data = response['Items']

In [2]:
import pandas as pd
df = pd.DataFrame(data)
print(df)

             billboard_audience_segment_id               count        date  \
0      000318d8446f9f979e0aade43c5a7889134                 1.0  2018-12-31   
1      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-01-07   
2      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-03-11   
3      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-03-18   
4      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-03-25   
5      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-04-01   
6      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-04-08   
7      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-04-15   
8      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-04-22   
9      000318d8446f9f979e0aade43c5a7889134                 1.0  2019-04-29   
10     000318d8446f9f979e0aade43c5a7889134                 1.0  2019-05-06   
11     000318d8446f9f979e0aade43c5a7889134                 1.0  

In [11]:
X = df.index.factorize()[0].reshape(-1,1)
print("X: ", X)

y = df["count"].tolist()
y = [float(i) for i in y]
print("y: ", y)

X:  [[    0]
 [    1]
 [    2]
 ...
 [10210]
 [10211]
 [10212]]
y:  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.333333333333333, 4.666666666666667, 6.0, 4.0, 2.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 6.0, 4.0, 4.0, 4.0, 6.0, 6.0, 6.0, 6.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 30.0, 25.5, 21.0, 16.5, 12.0, 11.0, 10.0, 30.0, 9.0, 10.0, 7.0, 8.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 30.0, 20.0, 20.0, 20.0, 30.0, 30.0, 30.0, 30.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0, 24.0

In [15]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [16]:
def X_to_x_windows(X):
    x = []
    for i in range(len(X)):
        x.append(X[i:(i+1)])
    return np.array(x)



lstm.eval()
tensorX = Variable(torch.Tensor(X_to_x_windows(X)))

train_predict = sc.inverse_transform(lstm(tensorX).data.numpy())

NameError: name 'lstm' is not defined